In [2]:
# Given Data Loader in Siamese_Sample folder
# Train Libraries
from __future__ import print_function
import argparse, random, copy
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision import transforms as T
from torch.optim.lr_scheduler import StepLR
# from dataloader import train_test_split, FingerprintDataset, ImageTransform
# from model import SiameseNetwork
import json


# Model Libraries
import torch
import torch.nn as nn
import torchvision


# Dataloader libraries
import os
import random
import albumentations
import torch
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import numpy as np


class ContrastiveLoss(torch.nn.Module):
    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_dist = F.pairwise_distance(output1, output2, keepdim=True)

        loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_dist, 2) +
                                      (label) * torch.pow(torch.clamp(self.margin - euclidean_dist, min=0.0), 2))
        return loss_contrastive

def train(model, device, train_loader, optimizer, epoch, log_interval, dry_run):
    model.train()

    # we aren't using `TripletLoss` as the MNIST dataset is simple, so `BCELoss` can do the trick.
    criterion = nn.BCELoss()

    for batch_idx, (images_1, images_2, targets, _, _) in enumerate(train_loader):
        images_1, images_2, targets = images_1.to(device), images_2.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(images_1, images_2).squeeze()
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(images_1), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            if dry_run:
                break

def test(model, device, test_loader, test_batch_size):
    model.eval()
    test_loss = 0
    tot_correct = 0

    true_correct = 0

    tot_test_easy = 0
    correct_test_easy = 0

    tot_test_med = 0
    correct_test_med = 0

    tot_test_hard = 0
    correct_test_hard = 0



    # we aren't using `TripletLoss` as the MNIST dataset is simple, so `BCELoss` can do the trick.
    criterion = nn.BCELoss()

    with torch.no_grad():
        for (images_1, images_2, targets, _, altered_addr) in test_loader:
            images_1, images_2, targets = images_1.to(device), images_2.to(device), targets.to(device)
            outputs = model(images_1, images_2).squeeze() 
            test_loss += criterion(outputs, targets).sum().item()  # sum up batch loss
            ones = torch.ones(outputs.shape[0],).to(device)
            zeros = torch.zeros(outputs.shape[0],).to(device)
            pred = torch.where(outputs > 0.5, ones, zeros)  # get the index of the max log-probability
            correctPred = pred.eq(targets.view_as(pred)).sum().item()
            true_correct += correctPred
            print("alteredaddr length: ", len(altered_addr))
            for i in range(len(altered_addr)):
            # print("correctPred:", correctPred)
                print("prediction and targets", pred[i], targets[i])
                print("alteredaddr", altered_addr[i])
                if "Easy" in altered_addr[i]:
                    # print("EASY")
                    tot_test_easy+=1
                    if pred[i]==targets[i]:
                        correct_test_easy+=1
                        tot_correct+=1

                elif "Medium" in altered_addr[i]:
                    # print("MEDIUM")
                    tot_test_med+=1
                    if pred[i]==targets[i]:
                        correct_test_med+=1
                        tot_correct+=1

                elif "Hard" in altered_addr[i]:
                    # print("HARD")
                    tot_test_hard+=1
                    if pred[i]==targets[i]:
                        correct_test_hard+=1
                        tot_correct+=1

    test_loss /= len(test_loader.dataset)

    # for the 1st epoch, the average loss is 0.0001 and the accuracy 97-98%
    # using default settings. After completing the 10th epoch, the average
    # loss is 0.0000 and the accuracy 99.5-100% using default settings.
    print("TRUE CORRECT: ", true_correct)
    print('\nTest set: Average loss: {:.4f}, Total Accuracy: {}/{} ({:.0f})%, Easy Accuracy: {}/{} ({:.0f})%, Medium Accuracy: {}/{} ({:.0f})%, Hard Accuracy: {}/{} ({:.0f})%)\n'.format(
        test_loss, tot_correct, len(test_loader.dataset), 100. * tot_correct / len(test_loader.dataset), 
        correct_test_easy, tot_test_easy, 100 * correct_test_easy / tot_test_easy,
        correct_test_med, tot_test_med, 100 * correct_test_med / tot_test_med,
        correct_test_hard, tot_test_hard, 100 * correct_test_hard / tot_test_hard,
        ))
# class SiameseNetwork(nn.Module):
#     def __init__(self):
#         super(SiameseNetwork, self).__init__()

#         # convolutional neural network
#         self.cnn1 = nn.Sequential(
#             nn.Conv2d(1, 96, kernel_size=11, stride=4),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(3, stride=2),

#             nn.Conv2d(96, 256, kernel_size=5, stride=1),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(2, stride=2),

#             nn.Conv2d(256, 384, kernel_size=3, stride=1),
#             nn.ReLU(inplace=True)
#         )

#         # fully connected layers
#         self.fc1 = nn.Sequential(
#             nn.Linear(384, 1024),
#             nn.ReLU(inplace=True),

#             nn.Linear(1024, 256), 
#             nn.ReLU(inplace=True),

#             nn.Linear(256, 2)
#         )
    
#     def forward_once(self, x):
#         output = self.cnn1(x)
#         output = output.view(output.size()[0], -1)
#         output = self.fc1(output)
#         return output
    
#     def forward(self, input1, input2):
#         output1 = self.forward_once(input1)
#         output2 = self.forward_once(input2)
#         output = torch.cat((output1, output2), 1)

#         return output
## GIVEN CODE for SIAMESE NETWORK

class SiameseNetwork(nn.Module):
    """
        Siamese network for image similarity estimation.
        The network is composed of two identical networks, one for each input.
        The output of each network is concatenated and passed to a linear layer. 
        The output of the linear layer passed through a sigmoid function.
        `"FaceNet" <https://arxiv.org/pdf/1503.03832.pdf>`_ is a variant of the Siamese network.
        This implementation varies from FaceNet as we use the `ResNet-18` model from
        `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_ as our feature extractor.
        In addition, we aren't using `TripletLoss` as the MNIST dataset is simple, so `BCELoss` can do the trick.
    """
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        # get resnet model
        self.resnet = torchvision.models.resnet18(pretrained=False)

        # over-write the first conv layer to be able to read MNIST images
        # as resnet18 reads (3,x,x) where 3 is RGB channels
        # whereas MNIST has (1,x,x) where 1 is a gray-scale channel
        self.resnet.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        self.fc_in_features = self.resnet.fc.in_features
        
        # remove the last layer of resnet18 (linear layer which is before avgpool layer)
        self.resnet = torch.nn.Sequential(*(list(self.resnet.children())[:-1]))
        # print(self.resnet)

        # add linear layers to compare between the features of the two images
        self.fc = nn.Sequential(
            nn.Linear(self.fc_in_features * 2, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 1),
        )

        self.sigmoid = nn.Sigmoid()

        # initialize the weights
        self.resnet.apply(self.init_weights)
        self.fc.apply(self.init_weights)
        
    def init_weights(self, m):
        if isinstance(m, nn.Linear):
            torch.nn.init.xavier_uniform(m.weight)
            m.bias.data.fill_(0.01)

    def forward_once(self, x):
        output = self.resnet(x)
        output = output.view(output.size()[0], -1)
        return output

    def forward(self, input1, input2):
        # get two images' features
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)

        # concatenate both images' features
        output = torch.cat((output1, output2), 1)

        # pass the concatenation to the linear layers
        output = self.fc(output)

        # pass the out of the linear layers to sigmoid layer
        output = self.sigmoid(output)
        
        return output
    


def train_test_split(base_dir, train_ratio, num_people = 600, randomize = True, tests = None):

  # Splitting the train & test data according to the 'train_ratio' (ex. 0.9 : 0.1) 
  train_size = num_people * train_ratio
  test_size = num_people - train_size

  # Setting train_nums to be a list composed of 1 ~ 600
  train_nums = [i+1 for i in range(0,num_people)]
  # Setting test_nums to be a list composed of 600 zeros ([0,0,.....,0])
  test_nums = [0 for i in range(0, num_people)]

  train_list = []
  test_list = []

  altered_root = os.path.join(base_dir, "Altered")
  altered_list = ["Altered-Easy", "Altered-Medium", "Altered-Hard"]

  test_check = []

  # Randomly select people who will be used as test subjects
  if randomize :
    while len(test_check) < test_size:
      test = random.randint(1,num_people)
      if test not in test_check :
        test_check.append(test)
        test_nums[test-1] = test
    
    for i in range(0, len(train_nums)):
      train_nums[i] = train_nums[i]-test_nums[i]

    train_nums = set(train_nums)
    train_nums.remove(0)
    train_nums = list(train_nums)

    test_nums = set(test_nums)
    test_nums.remove(0)
    test_nums = list(test_nums)

    print("Num Train Data : ",len(train_nums))
    print("Num Test Data : ", len(test_nums))
  # If we pre-select test subjects, then just remove these from the train subjects
  # train subjects will initially contain all individuals (1 ~ 600)
  else :
    test_nums = tests
    for i in range(0, len(test_nums)):
      if test_nums[i] in train_nums:
        train_nums.remove(test_nums[i])
  
  # Let's go through the altered images and organize them depending on 
  # whether they can be used for training or testing of the model
  for altered in altered_list : 
    altered_dir = os.path.join(altered_root, altered)
    altered_imgs = os.listdir(altered_dir)
    
    for img in altered_imgs :
      person_num = int(img[:img.find('__')])
      if person_num in test_nums :
        test_img_path = os.path.join(altered_dir, img)
        test_list.append(test_img_path)
      else:
        train_img_path = os.path.join(altered_dir, img)
        train_list.append(train_img_path)

  # train_list & test_list are made of altered images
  return train_nums, train_list, test_nums, test_list

class ImageTransform():
  def __init__(self, mean, std):
    self.data_transform = transforms.Compose([
        transforms.Resize((100, 100)),
        # transforms.Resize((95, 95)),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
    ])

  def __call__(self, img):
    # print(type(img))
    return self.data_transform(img)
      
class ImageAugTransform_train():
  def __init__(self, mean, std):
    self.albumentations_transform_oneof = albumentations.Compose([
                                                                  albumentations.Resize(100, 100),
                                                                  albumentations.Normalize(mean=mean, std=std, max_pixel_value=255),
                                                                  albumentations.OneOf([
                                                                                        albumentations.HorizontalFlip(p=0.5),
                                                                                        albumentations.RandomRotate90(p=0.5)
                                                                                        ], p=0.5),
                                                                  albumentations.OneOf([
                                                                  albumentations.GaussNoise(p=0.5)
                                                                  ], p=0.5),
                                                                  albumentations.RandomBrightness(limit=0.2, always_apply=False, p=0.5)
                                                                  ])

  def __call__(self, img):
      return self.albumentations_transform_oneof(image=img)

# The FingerPrintDataset used to load into pytorch dataloader
class FingerprintDataset(Dataset):
  def __init__(self, data_nums, data_list, transform=None, transAug=None):
    self.data_nums = data_nums
    self.data_list = data_list
    self.transform = transform
    self.transAug = transAug
    
  
  def __getitem__(self, idx):
  
    real_dir = '../../SOCOFingBMP/Real'
    real_imgs = os.listdir(real_dir)

    # img will be a case from altered image
    img = self.data_list[idx]
    # find a corresponding name that can be used for identifying the real image
    img_name_real = img[img.rfind('/')+1:img.rfind('_')] + '.BMP'
    
    # This will set probability for creating matching or non-matching case at 50%
    print_match = idx % 2 == 0

    # This will set probability for applying augmentation at 50%
    apply_aug = random.randint(0,1)
    
    # print_match : 0 means non-matching fingerprints
    # print_match : 1 means matching fingerprints
    if print_match : # Define the directory for which we will find the matching real case (same individual)
      real_img_dir = os.path.join(real_dir, img_name_real)
    else :
      while True: # Define the directory for which we will find the non-matching real case (different individual)
        real_img_num = random.choice(self.data_nums)-1 
        img_name_real_unmatch = real_imgs[real_img_num]
        
        if img_name_real != img_name_real_unmatch:
          real_img_dir = os.path.join(real_dir, img_name_real_unmatch)
          break

    real_directory = real_img_dir
    # print(real_directory)
    altered_directory = img
    # print(altered_directory)
    real = Image.open(real_img_dir).convert("L")
    altered = Image.open(img).convert("L")
    
    real = self.transform(real)

    if self.transAug is None :
      altered = self.transform(altered)
    else : 
      if apply_aug :
        altered_np = np.array(altered)
        altered_augmented = self.transAug(altered_np)
        altered = Image.fromarray(altered_augmented['image'])
        altered = self.transform(altered)
      else : 
        altered = self.transform(altered)
    
    # label = torch.from_numpy(np.array([int(print_match)], dtype=np.long))
    label = torch.tensor(print_match, dtype=torch.float)
    # label = print_match
    # pairset = {"real": real, "altered": altered, "label": torch.from_numpy(np.array([int(print_match)], dtype=np.float32))}
    
    # return pairset
    return real, altered, label, real_directory, altered_directory

  def __len__(self):
    length = len(self.data_list)
    return length
from datetime import datetime
def makedir(path):
	if not os.path.exists(path):
		os.makedirs(path)

# logs the model you used into a text file. 
# More than welcome to modify this function to keep track of the results
def log():
    with open('all_levels_newDataloader.ipynb', 'r') as inputfile:
        textstr = inputfile.read()
        fn = datetime.now().strftime('%Y_%m_%d_%H_%M_%S') + ".txt"
        log_path = "./logs/"
        makedir(log_path)
        with open(log_path + fn, 'w') as outputfile:
            outputfile.write(textstr)
        return fn
def main():
    fn = log()
    # Training settings
    # parser = argparse.ArgumentParser(description='PyTorch Siamese network Example')
    # parser.add_argument('--batch-size', type=int, default=64, metavar='N',
    #                     help='input batch size for training (default: 64)')
    batch_size = 128
    # parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
    #                     help='input batch size for testing (default: 1000)')
    test_batch_size = 1000
    # parser.add_argument('--epochs', type=int, default=14, metavar='N',
    #                     help='number of epochs to train (default: 14)')
    epochs = 6
    # parser.add_argument('--lr', type=float, default=1.0, metavar='LR',
    #                     help='learning rate (default: 1.0)')
    lr = 0.1
    # parser.add_argument('--gamma', type=float, default=0.7, metavar='M',
    #                     help='Learning rate step gamma (default: 0.7)')
    gamma = 0.7
    # parser.add_argument('--no-cuda', action='store_true', default=False,
    #                     help='disables CUDA training')
    no_cuda = False
    # parser.add_argument('--no-mps', action='store_true', default=False,
    #                     help='disables macOS GPU training')
    no_mps = False
    # parser.add_argument('--dry-run', action='store_true', default=False,
    #                     help='quickly check a single pass')
    dry_run = False
    # parser.add_argument('--seed', type=int, default=1, metavar='S',
    #                     help='random seed (default: 1)')
    seed = 1
    # parser.add_argument('--log-interval', type=int, default=10, metavar='N',
    #                     help='how many batches to wait before logging training status')
    log_interval = 10
    # parser.add_argument('--save-model', action='store_true', default=False,
    #                     help='For Saving the current Model')
    save_model = True
    # args = parser.parse_args()
    
    use_cuda = not no_cuda and torch.cuda.is_available()

    torch.manual_seed(seed)

    device = torch.device("cuda:0")

    train_kwargs = {'batch_size': batch_size}
    test_kwargs = {'batch_size': test_batch_size}
    if use_cuda:
        cuda_kwargs = {'num_workers': 1,
                       'pin_memory': True,
                       'shuffle': True}
        train_kwargs.update(cuda_kwargs)
        test_kwargs.update(cuda_kwargs)


    mean = (0.5,) #q1
    std = (0.5,)

    # Change the directory to where SOCOFing is located. 
    # Direcotories should look as the following
    # SOCOFing
    # -- Altered
    #    -- ...
    # -- Real
    #    -- ...
    base_dir = '../../SOCOFingBMP/'

    nums = {}
    train_nums, train_list, test_nums, test_list = train_test_split(base_dir, 0.7)
    print("Train List Length : ", len(train_list))
    print("Test List Length : ", len(test_list))

    nums['train'] = train_nums
    nums['test'] = test_nums
    
    # state_dict save file name : siamese_network_train%_test%_lr.pt
    outpath = 'train_test_split_{fn}.json'

    with open(outpath, 'w') as f:
        json.dump(nums, f)


    train_dataset = FingerprintDataset(train_nums, train_list, transform=ImageTransform(mean,std), transAug=None)
    test_dataset = FingerprintDataset(test_nums, test_list, transform=ImageTransform(mean,std))
    
    train_loader = torch.utils.data.DataLoader(train_dataset,**train_kwargs)
    test_loader = torch.utils.data.DataLoader(test_dataset, **test_kwargs)

    model = SiameseNetwork().to(device)
    optimizer = optim.Adadelta(model.parameters(), lr=lr)
    # optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum = 0.9, weight_decay = 1e-6) 

    scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
    for epoch in range(1, epochs + 1):
        train(model, device, train_loader, optimizer, epoch, log_interval, dry_run)
        test(model, device, test_loader, test_batch_size)
        scheduler.step()
        if epoch % 5==0 and save_model:
            # state_dict save file name : siamese_network_train%_test%_lr.pt
            fileName = str(f'Siamese_network_Date:{fn}__Epoch:{epoch}.pt')
            torch.save(model.state_dict(), fileName)




if __name__ == '__main__':
    main()

Num Train Data :  420
Num Test Data :  180
Train List Length :  34496
Test List Length :  14774


/tmp/ipykernel_32075/1014197067.py:225: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(m.weight)


Train Epoch: 1 [0/34496 (0%)]	Loss: 0.798725
Train Epoch: 1 [1280/34496 (4%)]	Loss: 0.665253
Train Epoch: 1 [2560/34496 (7%)]	Loss: 0.545991
Train Epoch: 1 [3840/34496 (11%)]	Loss: 0.591728
Train Epoch: 1 [5120/34496 (15%)]	Loss: 0.310976
Train Epoch: 1 [6400/34496 (19%)]	Loss: 0.249553
Train Epoch: 1 [7680/34496 (22%)]	Loss: 0.348423
Train Epoch: 1 [8960/34496 (26%)]	Loss: 0.261792
Train Epoch: 1 [10240/34496 (30%)]	Loss: 0.148376
Train Epoch: 1 [11520/34496 (33%)]	Loss: 0.226249
Train Epoch: 1 [12800/34496 (37%)]	Loss: 0.117022
Train Epoch: 1 [14080/34496 (41%)]	Loss: 0.168114
Train Epoch: 1 [15360/34496 (44%)]	Loss: 0.159113
Train Epoch: 1 [16640/34496 (48%)]	Loss: 0.152216
Train Epoch: 1 [17920/34496 (52%)]	Loss: 0.201050
Train Epoch: 1 [19200/34496 (56%)]	Loss: 0.109179
Train Epoch: 1 [20480/34496 (59%)]	Loss: 0.138588
Train Epoch: 1 [21760/34496 (63%)]	Loss: 0.111039
Train Epoch: 1 [23040/34496 (67%)]	Loss: 0.135633
Train Epoch: 1 [24320/34496 (70%)]	Loss: 0.084352
Train Epoch: 1

KeyboardInterrupt: 